# It couldn't be inspected
Because it was quantilized dynamically, so it will not add a quan layer into model,
we couldn't inspect it.

In [1]:
import torch
import torch.quantization


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "/Users/kki/anaconda3/envs/d2l/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/Users/kki/anaconda3/envs/d2l/lib/python3.9/runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "/Users/kki/anaconda3/envs/d2l/lib/python3.9/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kki/anaconda3/envs/d2l/lib/python3.9/site-packages/traitlets/config/application.py", line 1075, in launch_instance
 

In [2]:
# 定义一个简单的全连接层模型
class MyModel(torch.nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = torch.nn.Linear(10, 2)

    def forward(self, x):
        print("量化前的输入: ", x)  # 打印量化前的浮点输入
        x = self.fc(x)
        print("量化后的输出: ", x)  # 打印计算后的输出
        return x

In [3]:
# 创建模型实例并切换到评估模式
model = MyModel()
model.eval()

MyModel(
  (fc): Linear(in_features=10, out_features=2, bias=True)
)

In [4]:
# 使用动态量化，量化模型的Linear层
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8  # 对Linear层量化为int8
)

In [7]:
# 输入张量 (浮点数)
inputs = torch.randn(1, 10)
inputs

tensor([[-0.8254, -1.0313,  1.6480, -0.6744,  0.6413, -0.8191,  1.5195, -1.7864,
         -0.4088,  0.5763]])

In [8]:
# 推理并验证输入
output = quantized_model(inputs)

量化前的输入:  tensor([[-0.8254, -1.0313,  1.6480, -0.6744,  0.6413, -0.8191,  1.5195, -1.7864,
         -0.4088,  0.5763]])
量化后的输出:  tensor([[ 0.5230, -1.3557]])


In [9]:
print(quantized_model)

MyModel(
  (fc): DynamicQuantizedLinear(in_features=10, out_features=2, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
)


In [10]:
print(model)

MyModel(
  (fc): Linear(in_features=10, out_features=2, bias=True)
)
